### Py_12 Введение

Задание 1.1 Преобразуйте столбец Date в формат datetime и выделите квартал (quarter) продажи объектов недвижимости. Найдите второй по популярности квартал продажи. В качестве ответа запишите число объектов, проданных в этом квартале.

In [42]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df.head()
#melb_df.info()

melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
quarter_best = melb_df['Date'].dt.quarter
print(quarter_best.value_counts().iloc[1])


4359


Задание 1.2 Преобразуйте все столбцы, в которых меньше 150 уникальных значений, в тип данных category, исключив из преобразования столбцы Date, Rooms, Bedroom, Bathroom, Car.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных category

In [57]:
cols_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
max_unique_count = 150
for i in melb_df.columns:
    if melb_df[i].nunique() < max_unique_count and i not in cols_exclude:
        melb_df[i] = melb_df[i].astype('category')
        
display(melb_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Suburb           13580 non-null  category      
 1   Rooms            13580 non-null  int64         
 2   Type             13580 non-null  category      
 3   Price            13580 non-null  float64       
 4   Method           13580 non-null  category      
 5   SellerG          13580 non-null  category      
 6   Date             13580 non-null  datetime64[ns]
 7   Distance         13580 non-null  float64       
 8   Postcode         13580 non-null  int64         
 9   Bedroom          13580 non-null  int64         
 10  Bathroom         13580 non-null  int64         
 11  Car              13580 non-null  int64         
 12  Landsize         13580 non-null  float64       
 13  BuildingArea     13580 non-null  float64       
 14  CouncilArea      12211 non-null  categ

None

# Сортировка sort_values()

Сортируем по столбцу 'Price' по возрастанию сверху вниз

In [ ]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_fe.csv')

melb_df.sort_values(by='Price').head(10)

Сортировка по столбцу 'Date' по убыванию сверху-вниз

In [ ]:
import pandas as pd
melb_df = pd.read_csv('data/melb_data_fe.csv')

melb_df.sort_values(by='Date', ascending=False)

Сортируем таблицу сначала по возрастанию расстояния от центра города (Distance), а затем — по возрастанию цены объекта (Price)

In [ ]:
melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath, у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8. Результат отсортируем по дате продажи (Date) в порядке возрастания, а после проведём сортировку по убыванию коэффициента соотношения площадей. Также обновим старые индексы на новые, установив параметр ignore_index на True. Для наглядности результата выберем из таблицы только столбцы Data и AreaRatio

In [ ]:
mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df.sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

Задание 2.2 Произведите сортировку столбца AreaRatio по убыванию. При этом индексы полученной таблицы замените на новые. Какое значение площади здания находится в строке 1558? Ответ округлите до целого числа.

In [44]:
melb_df.sort_values(by=['AreaRatio'], ascending=False, ignore_index=True). loc[1558, ['BuildingArea']]


BuildingArea    126.0
Name: 1558, dtype: object

Задание 2.3 Найдите таунхаусы (Type) с количеством жилых комнат (Rooms) больше 2. Отсортируйте полученную таблицу сначала по возрастанию числа комнат, а затем по убыванию средней площади комнат (MeanRoomsSquare). Индексы таблицы замените на новые. Какая цена будет у объекта в строке 18? Ответ запишите в виде целого числа.

In [50]:
mask_1 = melb_df['Type'] == 'townhouse'
mask_2 = melb_df['Rooms'] > 2

# Внимание!!! начинаем с int что бы сразу преобразовать к целому числу
int(melb_df [mask_1 & mask_2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
). loc[18, 'Price'])

1300000

### Метод GroupBy( ) - группировка

Применяем агрегирующую функцию среднего к результату работы groupby(). В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type):

In [ ]:
melb_df.groupby(by='Type', as_index=False).mean() # as_index=True добавляем индекс первым столбом

Как правило, нам не нужна информация обо всех столбцах, поэтому агрегирующие методы можно применять только к интересующему нас столбцу. Например, давайте сравним средние цены на объекты в зависимости от их типа:

In [ ]:
melb_df.groupby('Type')['Price'].mean()

Найдем какие регионы (Regionname) наиболее удалены от центра Мельбурна.<br>
1. найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона
2. результат отсортируем по убыванию расстояния:

In [ ]:
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ

Построим таблицу для анализа продаж по месяцам.
1. найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price)
2. сгруппированных по номеру месяца продажи (MonthSale)
3. результат отсортируем по количеству продаж в порядке убывания:

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'max']
).sort_values(by='count', ascending=False)

Полная информация обо всех основных статистических характеристиках внутри каждой группы, вы можете воспользоваться методом agg(), передав в качестве его параметра строку 'describe':

In [ ]:
melb_df.groupby('MonthSale')['Price'].agg('describe')

Подсчёт числа уникальных значений.
Передача функции set в agg() - получим множество из агентств недвижимости, которые работают в каждом из регионов.

In [ ]:
melb_df.groupby('Regionname')['SellerG'].agg(['nunique', set])

Задание 3.1 Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.

In [ ]:
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

Задание 3.2 Какой регион имеет наименьшую протяжённость по географической широте (Lattitude)?
Для ответа на этот вопрос рассчитайте стандартное отклонение широты для каждого региона.
В качестве ответа запишите название этого региона.

In [ ]:
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

Задание 3.3 Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?
<br>Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
<br>Не забудьте перевести даты в формат datetime.

In [ ]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
month_may = melb_df['Date'].dt.month
#print(month_may)
period_1 = pd.to_datetime('2017-05-01')
period_2 = pd.to_datetime('2017-09-01')
mask = (period_1 <= melb_df['Date']) & (melb_df['Date'] <= period_2)
melb_df[mask].groupby(['SellerG'])['Price'].sum().sort_values(ascending=True)

### Сводные таблицы

In [ ]:
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

Сводная таблица: В качестве параметра values укажем столбец Price, в качестве индексов сводной таблицы возьмём Rooms, а в качестве столбцов — Type. Агрегирующую функцию оставим по умолчанию (среднее). Дополнительно заменим пропуски в таблице на значение 0. Финальный результат для наглядности вывода округлим с помощью метода round() до целых.

In [ ]:
melb_df.pivot_table(
    values = 'Price',
    index = 'Rooms',
    columns = 'Type',
    fill_value = 0
).round()

Построим сводную таблицу, в которой строками будут являться названия регионов (Regionname), а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), который равен 1, если день был выходным, и 0 — в противном случае. В качестве значений сводной таблицы возьмём количество продаж.

In [ ]:
melb_df.pivot_table(
    values = 'Price',
    index = 'Regionname',
    columns = 'Weekend',
    aggfunc = 'count'
)

Найдём, как зависит средняя и медианная площадь участка (Landsize) от типа объекта (Type) и его региона (Regionname). Чтобы посмотреть несколько статистических параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. Построим такую сводную таблицу, где пропущенные значения заменим на 0:

In [ ]:
melb_df.pivot_table(
    values = 'Landsize',
    index = 'Regionname',
    columns = 'Type',
    aggfunc = ['median', 'mean'],
    fill_value = 0
)

Построим таблицу, в которой по индексам будут располагаться признаки метода продажи (Method) и типа объекта (Type), по столбцам — наименование региона (Regionname), а на пересечении строк и столбцов будет стоять медианная цена объекта (Price):

In [ ]:
melb_df.pivot_table(
    values = 'Price',
    index = ['Method', 'Type'],
    columns = 'Regionname',
    aggfunc = 'mean',
    fill_value = 0
)

### Доступ к данным таблицы

In [ ]:
pivot = melb_df.pivot_table(
    values = 'Landsize',
    index = 'Regionname',
    columns = 'Type',
    aggfunc = ['median', 'mean'],
    fill_value = 0
)

#просмотр столбцов и иерархии
pivot.columns

# Среднее значение площадей участков с домами типа unit - последовательно обращаемся к столбам, начиная с верхнег уровня и вниз по иерархии
display(pivot['mean']['unit'])

#Фильтрация. Вывод, где средняя площадь участков с домами типа house меньше их медианной площади
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

Создание сводных (мультииндексных) таблиц вручную
Передаём в качестве индексов Series вложенный список, где первый список задаёт внешний уровень вложенности, а второй список — внутренний уровень вложенности. Значения Series — случайные числа от 0 до 1, сгенерированные функцией np.random.rand()

In [ ]:
import pandas as pd
import numpy as np

mser = pd.Series(
    np.random.rand(8),
	index=[['white','white','white','blue','blue','red','red','red'], 
           ['up','down','right','up','down','up','down','left']])
display(mser)

#Если посмотреть на индексы Series, можно увидеть, что они являются мультииндексами:
print(mser.index)

Создание сводных (мультииндексных) таблиц вручную
Аналогично создаются DataFrame со вложенными признаками (вложенными столбцами) — для этого вложенный список передаётся в параметр columns при инициализации таблицы:

In [ ]:
mframe = pd.DataFrame(
    np.random.randn(16).reshape(4,4),
    index=[['white','white','red','red'], ['up','down','up','down']],
    columns=[['pen','pen','paper','paper'],[1,2,1,2]]
)
display(mframe)

Задание 4.2 Составьте сводную таблицу, которая показывает зависимость медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) и количества жилых комнат в доме (Rooms). Для какой комбинации признаков площадь здания наибольшая?
В качестве ответа запишите эту комбинацию (тип здания, число комнат) через запятую, без пробелов.

In [ ]:
pivot = melb_df.pivot_table(
    values = 'BuildingArea',
    index = 'Type',
    columns = 'Rooms',
    aggfunc = ['median'],
    fill_value = 0
)

display(pivot)

Задание 4.3 Составьте сводную таблицу, которая показывает зависимость средней цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания (Type).
Во вновь созданной таблице найдите агентство, у которого средняя цена для зданий типа unit максимальна. В качестве ответа запишите название этого агентства.

In [ ]:
pivot = melb_df.pivot_table(
    values = 'Price',
    index='SellerG',
    columns='Type',
    aggfunc='mean',
    fill_value=0
)

result = pivot['unit'].max()
display(pivot[pivot['unit'] == result])




### Объединение DataFrame: знакомимся с новыми данными

Задание 5.2 Сколько уникальных фильмов представлено в таблице movies?

In [ ]:
import pandas as pd
import numpy as np

movies_df = pd.read_csv('data/movies.csv')
movies_df.head()
movies_df.info()

Задание 5.3 Сколько уникальных пользователей в таблице ratings1?

In [76]:
ratings1_df = pd.read_csv('data/ratings1.csv')
ratings1_df.head()
ratings1_df['userId'].nunique()
#ratings1_df.info()

274

Задание 5.4 В каком году было выставлено больше всего оценок?
Для ответа на этот вопрос используйте таблицу dates.

In [ ]:
dates_df = pd.read_csv('data/dates.csv')
dates_df['date'] = pd.to_datetime(dates_df['date'])
dates_df.head()
result = dates_df['date'].dt.year.mode()
result_2 = dates_df['date'].dt.year.value_counts()
display(result, result_2)

#dates_df.mode()

ЗАДАНИЕ 6.3 
В ваше распоряжение предоставлена директория "./Root/users/". В данной директории содержатся csv-файлы, в каждом из которых хранится информация об идентификаторах пользователей (user_id) и ссылки на их фотографии (photo_url). <br>
При проверке в директории может быть сколько угодно файлов.<br>
Вам необходимо написать функцию concat_user_files(path), параметром которой является path — путь до директории. Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его.<br> 
Список названий всех файлов, находящихся в директории, вы можете получить с помощью функции os.listdir(path) из модуля os (модуль уже импортирован в файле main.py). Например, для директории "./Root/users/" результатом работы функции будет список:<br>
print(os.listdir('./Root/users/'))<br>
['users2.csv', 'users1.csv', 'users3.csv']<br>
Примечание. Модуль os позволяет работать с операционной системой компьютера прямо из кода. Подробнее о нем вы можете почитать здесь.
Отсортируйте этот список, прежде чем производить объединение файлов.<br>
Когда вы получите отсортированный список, вам останется только прочитать все csv-файлы из списка в цикле и объединить прочитанные таблицы между собой.<br>
Однако обратите внимание, что метод os.listdir() возвращает только список имён файлов в указанной директории, а при чтении файла необходимо указывать полный путь до него. То есть путь для чтения будет таким:<br>
'./Root/users/{file_name}'<br>
Не забудьте обновить индексы результирующей таблицы после объединения.<br>
Учтите, что на тестовом наборе файлов в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться.

In [ ]:
import pandas as pd
import os

"""
    Вам необходимо написать функцию concat_user_files(path), параметром которой является path - путь до директории. 
    Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его. 
    Не забудьте обновить индексы результирующей таблицы после объединения.
    Учтите тот момент, что в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться. 
    """

def concat_users_files(path):
    data = pd.DataFrame()
    file_names = os.listdir(path)
    file_names.sort()
    for file in file_names:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data], axis=0, ignore_index=True)
    data = data.drop_duplicates()
    return data

if __name__ == '__main__':
    data = concat_users_files('./Root/users/')

ЗАДАНИЕ 7.5
Даны две таблицы: items_df, в которой содержится информация о наличии товаров на складе, и purchase_df — с данными о покупках товаров.<br>

items_df = pd.DataFrame({<br>
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], <br>
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],<br>
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]<br>
})<br>
purchase_df = pd.DataFrame({<br>
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],<br>
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], <br>
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]<br>
})<br>
Информация в таблицах представлена в виде следующих столбцов:<br>

item_id — идентификатор модели;<br>
vendor — производитель модели;<br>
stock_count — имеющееся на складе количество данных моделей (в штуках);<br>
purchase_id — идентификатор покупки;<br>
price — стоимость модели в покупке.<br>
Сформируйте DataFrame merged, в котором в результате объединения purchase_df и items_df останутся модели, которые учтены на складе и имели продажи.<br>
Из таблицы merged найдите суммарную выручку, которую можно было бы получить от продажи всех товаров, которые учтены на складе и имели продажи. <br>Результат занесите в переменную income.

In [ ]:
merged = items_df.merge(purchase_df, how='inner', on='item_id')
income = (merged['price'] * merged['stock_count']).sum()